In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision
from attack import attack, test_model


In [6]:
import re
def parse_layer(layer):
    reg = re.compile("\.\d+\.")
    finded = reg.findall(layer)
    if len(finded) == 0:
        pass
    else:
        for f in finded:
            f = f[1:-1]
            layer = layer.replace(f".{f}.", f"[{f}].")
    return layer

In [2]:
from datasets import load_cifar10, load_cifar100
from models.resnet import load_cifar10_resnet50, load_cifar100_resnet50
model = load_cifar10_resnet50()


In [3]:
all_param_names = list()
for name, param in model.named_parameters():
    all_param_names.append(name)

In [4]:
train_loaders, test_dataloaders,train_dataloader_all, test_dataloader_all = load_cifar10()
all_totals = list()
# for i in range(10):
all_totals.append(attack(train_dataloader_all, all_param_names, load_cifar10_resnet50, alpha=0.0001))


Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 196/196 [01:22<00:00,  2.38it/s]


0.0005741860693320632


100%|██████████| 196/196 [01:15<00:00,  2.61it/s]


0.49847307960510256


...

100%|██████████| 196/196 [01:19<00:00,  2.45it/s]


60.13131256835938


100%|██████████| 196/196 [01:21<00:00,  2.40it/s]


228.0817545703125


d:\Documents\GitHub\Neural-importance\P1模型训练+归因测试\参数归因Cifar10不分label\attack.py:71: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  layer_totals = np.array(layer_totals)
d:\Documents\GitHub\Neural-importance\P1模型训练+归因测试\参数归因Cifar10不分label\utils.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = np.array(x)


In [5]:
all_totals


[{'conv1.weight': array([[[[  4.5681806 ,   1.7417259 ,   1.5368824 ],
           [  4.5582333 ,   2.9617667 ,   5.6790276 ],
           [  7.7801304 ,   6.7803698 ,   7.075375  ]],
  
          [[  2.4021156 ,  -0.56369966,  -0.92677015],
           [  4.473156  ,   1.9707037 ,   5.8066607 ],
           [  4.3532887 ,   6.8055997 ,   7.705123  ]],
  
          [[ -6.222246  ,  -9.012195  ,  -8.843936  ],
           [ -3.820074  ,  -4.778753  ,  -2.7443106 ],
           [ -6.267524  ,  -3.6586683 ,  -0.21875818]]],
  
  
         [[[ -9.317438  , -17.825832  , -35.982216  ],
           [-13.355007  , -12.439405  , -26.864311  ],
           [ -9.941201  ,  -2.6939464 ,  -7.2986884 ]],
  
          [[ -6.232153  , -12.051174  , -30.450233  ],
           [-13.44108   , -11.594536  , -24.027088  ],
           [-11.483533  ,  -2.0339868 ,  -6.6920195 ]],
  
          [[ -4.920095  , -11.401877  , -28.43239   ],
           [-11.634388  ,  -8.260717  , -23.268759  ],
           [-12.9081545 ,

In [37]:
thre = 0.4
net = load_cifar10_resnet50()
layer_remove = dict()
for layer in all_param_names:
    layer_remove[layer] = None
for i in range(len(all_totals)):
    totals = all_totals[i]
    totals = [totals[layer] for layer in all_param_names]
    layer_weights = [eval("net." + parse_layer(layer) + ".cpu().detach().numpy()")
                     for layer in all_param_names]
    combine = [np.abs(total * weight) for total, weight in zip(totals, layer_weights)]
    combine = np.array(combine)
    combine_flatten = np.concatenate([combine_.flatten() for combine_ in combine],axis=0)
    threshold = np.sort(combine_flatten)[::-1][int(len(combine_flatten) * thre)]
    for idx,layer in enumerate(all_param_names):
        if layer_remove[layer] is None:
            layer_remove[layer] = combine[idx] > threshold
        else:
            t = combine[idx] > threshold
            layer_remove[layer] = layer_remove[layer] | t

C:\Users\Zhang\AppData\Local\Temp\ipykernel_31012\911857792.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  combine = np.array(combine)


In [38]:
temp = 0
all_num = 0
for layer in layer_remove:
    temp += layer_remove[layer].sum()
    all_num += layer_remove[layer].size
    print(layer, layer_remove[layer].mean())

conv1.weight 0.9427083333333334
bn1.weight 1.0
bn1.bias 1.0
layer1.0.conv1.weight 0.75927734375
layer1.0.bn1.weight 0.984375
layer1.0.bn1.bias 0.953125
layer1.0.conv2.weight 0.6896701388888888
layer1.0.bn2.weight 1.0
layer1.0.bn2.bias 0.96875
layer1.0.conv3.weight 0.69451904296875
layer1.0.bn3.weight 0.9375
layer1.0.bn3.bias 0.90625
layer1.0.shortcut.0.weight 0.7081298828125
layer1.0.shortcut.1.weight 0.921875
layer1.0.shortcut.1.bias 0.90625
layer1.1.conv1.weight 0.70196533203125
layer1.1.bn1.weight 0.96875
layer1.1.bn1.bias 0.890625
layer1.1.conv2.weight 0.7052137586805556
layer1.1.bn2.weight 1.0
layer1.1.bn2.bias 0.921875
layer1.1.conv3.weight 0.64849853515625
layer1.1.bn3.weight 0.89453125
layer1.1.bn3.bias 0.9375
layer1.2.conv1.weight 0.64678955078125
layer1.2.bn1.weight 0.984375
layer1.2.bn1.bias 0.890625
layer1.2.conv2.weight 0.60986328125
layer1.2.bn2.weight 1.0
layer1.2.bn2.bias 0.984375
layer1.2.conv3.weight 0.63153076171875
layer1.2.bn3.weight 0.87109375
layer1.2.bn3.bias 0.

In [39]:
temp / all_num

0.3999999234721274

In [40]:
with torch.no_grad():
    net = load_cifar10_resnet50()
    correct, all = test_model(net, test_dataloader_all)
    print("原始准确率", correct / all)


原始准确率 0.954


In [41]:
with torch.no_grad():
    net = load_cifar10_resnet50()
    for layer in all_param_names:
        layer_ = parse_layer(layer)
        if len(eval("net." + layer_ + ".shape")) == 2:
            exec("net." + layer_ + "[~layer_remove[layer],:] = 0")
        else:
            exec("net." + layer_ + "[~layer_remove[layer]] = 0")
    # for layer in ["layer3[0].conv2"]:
    #     exec("net." + layer + ".weight[~remove] = 0")
    correct, all = test_model(net, test_dataloader_all)
    print("现在准确率", correct / all)


现在准确率 0.8378


In [42]:
with torch.no_grad():
    net = load_cifar10_resnet50()
    for layer in all_param_names:
        layer_ = parse_layer(layer)
        keep_rate = layer_remove[layer].sum() / layer_remove[layer].size
        weight_flatten = eval("net." + layer_ + ".cpu().detach().numpy()").flatten()
        threshold = np.sort(weight_flatten)[int(len(weight_flatten) * (1 - keep_rate))]
        if len(eval("net." + layer_ + ".shape")) == 2:
            exec("net." + layer_ + "[eval('net.' + layer_ + '.cpu().detach().numpy()') < threshold,:] = 0")
        else:
            exec("net." + layer_ + "[eval('net.' + layer_ + '.cpu().detach().numpy()') < threshold] = 0")
    correct, all = test_model(net, test_dataloader_all)
    print("去掉最大准确率", correct / all)
    # for layer in ["layer3[0].conv2"]:
    #     threshold = np.sort(eval("net." + layer + ".weight.cpu().detach().numpy()").flatten())[
    #         int(len(eval("net." + layer +
    #             ".weight.cpu().detach().numpy()").flatten()) * (1 - keep_rate))
    #     ]
    #     exec("net." + layer +
    #          ".weight[eval('net.' + layer + '.weight.cpu().detach().numpy()') < threshold] = 0")
    # correct, all = test_model(net, test_dataloader_all)
    # print("现在准确率", correct / all)


去掉最大准确率 0.1
